# Read Initial Data

In [114]:
import pandas as pd
time = []
open = []
high = []
low = []
close = []
volume = []

def read_df(file_name):

    df = pd.read_feather(file_name)
    df = df.dropna()    # Dropping NA values, so that talib works correctly

    global time
    time = df['time'].to_numpy()

    global open
    open = df['open'].to_numpy()

    global high
    high = df['high'].to_numpy()

    global low
    low = df['low'].to_numpy()

    global close
    close = df['close'].to_numpy()

    global volume
    volume = df['volume'].to_numpy()

    return

read_df("data-BNBUSDT1h.feather")

# Create Indicators

In [115]:
import sys
numpy.set_printoptions(threshold=sys.maxsize)
import numpy
import talib    # https://github.com/mrjbq7/ta-lib

ma_direction = talib.EMA(close, timeperiod=24)
print(ma_direction)

[         nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan   8.25960833   8.28524767
   8.29843585   8.32136099   8.32754011   8.3211129    8.31514387
   8.31119636   8.31110065   8.3102126    8.32161959   8.33361802
   8.34088058   8.34961013   8.37684132   8.39277402   8.4065521
   8.41993993   8.43835273   8.45001251   8.48511551   8.51405827
   8.53693361   8.56989892   8.58150701   8.59790645   8.60616193
   8.59926098   8.6068481    8.60950025   8.60554023   8.60964901
   8.61046909   8.61162356   8.60267768   8.58803946   8.58894831
   8.58772044   8.60394281   8.60618738   8.60745239   8.6168082
   8.62746354   8.65248246   8.66460386   8.67703556   8.69567271
   8.71993889   8.76666378   8.78133068   8.78286423   8.81823509
   8.8514562

# Generate Trades

In [116]:
# Take idea from Zeus